In [6]:
# update your password
from mlutils import storage
storage.update_secret("password", "YOUR PASSWORD HERE")

In [1]:
from sharepoint_inhome import get_data, clean_data, create_log, send_email, to_bq, main, create_excel_file

In [2]:
df = get_data()

In [3]:
df.head()

,Store,Start Date,End date,FY25 exapansion wave
0,228,2024-07-24,NaN,FY25 Wave 1.5 Launch
1,371,2024-07-24,NaN,FY25 Wave 1.5 Launch
2,963,2024-07-24,NaN,FY25 Wave 1.5 Launch
3,6963,2024-07-24,NaN,FY25 Wave 1.5 Launch
4,5987,2024-07-24,NaN,FY25 Wave 1.5 Launch


In [4]:
# manually add a duplicated row
df.loc[len(df.index)] = ['500', '2022-09-21', '2024-08-06', '1', 'FY25 Wave 1 Capacity Expansion']

In [5]:
# manually add a missing row
df.loc[len(df.index)] = ['500', None, '2024-08-06', '1', 'FY25 Wave 1 Capacity Expansion']

In [4]:
missing, duplicated = clean_data(df)

In [5]:
missing

,Store,Start_Date,End_Date,Fy25_Exapansion_Wave


In [6]:
duplicated

,Store,Start_Date,End_Date,Fy25_Exapansion_Wave


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 0 to 314
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Store                 315 non-null    int32 
 1   Start_Date            315 non-null    object
 2   End_Date              315 non-null    object
 3   Fy25_Exapansion_Wave  315 non-null    object
dtypes: int32(1), object(3)
memory usage: 11.1+ KB


In [9]:
# test create_excel_file function
create_excel_file({'Missing': missing, 'Duplicated': duplicated})

C:\Users\y0q00wv\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [11]:
# test success email notification
log_body_with_attachment = "\nThere're missing and/or duplicated values in excel file, list in attachment, those records are not uploaded to BigQuery table."

bq_res = to_bq(df)
is_success, log_body = bq_res
# send log based on upload to BigQuery result
if is_success:
    attachment = create_excel_file({'Missing': missing, 'Duplicated': duplicated})
    if attachment is not None:
        log_body += log_body_with_attachment
        msg = create_log(log_body, is_failed=False, attachment=attachment)
        send_email(msg)
    else:
        msg = create_log(log_body, is_failed=True)
        send_email(msg)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [9]:
# test fail email notification
log_body = "TEST on FAIL"
msg = create_log(log_body, is_failed=True)
send_email(msg)

In [ ]:
run sharepoint_inhome.py --config_van_count.json